In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import random
import pickle
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import pandas as pd
import os
from sklearn.decomposition import PCA
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import keras
import sys

Using TensorFlow backend.


In [25]:
with open('/home/sxz/data/geolife_Data/paper2_trips_motion_features_NotFixedLength_woOutliers.pickle', 'rb') as f:
    a2 , b2 = pickle.load(f)
print(len(b2))
print(np.shape(b2[2]))

1141
(7, 2194)


In [26]:
with open('/home/sxz/data/geolife_Data/my_trips_motion_features_NotFixedLength_woOutliers.pickle', 'rb') as f:
    a2 , b2 = pickle.load(f)
print(len(b2))
print(np.shape(b2[2]))

739
(7, 110)


In [5]:
with open('/home/sxz/data/geolife_Data/My_data_for_DL_kfold_dataset_RL.pickle', 'rb') as f:
    a1 , b1 = pickle.load(f)
print(np.shape(a1[1][1]))
print(np.shape(a1[1][2]))
print(np.shape(b1))
print(a1[1][0][0])

(14444,)
(3609, 1, 248, 4)
(39237, 1, 248, 4)
[[[1.06085324e-01 5.41475412e-01 3.74140462e-02 5.96423367e-06]
  [1.37057485e-01 4.48904787e-01 3.90502391e-02 3.97605655e-06]
  [9.83287495e-02 4.74498005e-01 3.91216857e-02 1.51220441e-04]
  [7.88704985e-02 5.05250896e-01 3.86684899e-02 1.97519702e-04]
  [2.15105707e-01 4.17772254e-01 3.90276604e-02 1.53010691e-05]
  [1.52935993e-01 4.41734761e-01 3.86775909e-02 9.73760864e-06]
  [1.08808588e-01 4.40415698e-01 3.95906345e-02 3.97455391e-07]
  [6.36881795e-02 5.05035468e-01 3.88436803e-02 5.96147103e-07]
  [6.72230225e-02 5.15711210e-01 4.28442400e-02 1.58969864e-06]
  [2.16018942e-01 4.77027323e-01 4.01829394e-02 1.54945345e-05]
  [1.98465060e-01 5.84423024e-01 3.76694573e-02 1.78597268e-06]
  [2.61774482e-01 5.10297859e-01 3.78072418e-02 7.94269122e-06]
  [2.69271641e-01 4.46123053e-01 3.95248688e-02 8.33769406e-06]
  [2.28448566e-01 5.05993311e-01 3.83590955e-02 7.34312042e-06]
  [2.32704617e-01 4.81672823e-01 3.89096348e-02 1.07194226

In [18]:
kfold_dataset = [[] for _ in range(5)]

In [19]:
for i in range(5):
    a = a1[round(0.2*(i)*len(a1)):round((0.2*(i+1)*len(a1)))]
    kfold_dataset[i].append(a)
    print(np.shape(kfold_dataset[i][0]))

(1, 5)
(1, 5)
(1, 5)
(1, 5)
(1, 5)


In [14]:
a1[0][3]

IndexError: index 3 is out of bounds for axis 0 with size 1

In [6]:
np.shape(a1[0][0])

(31, 8)

In [7]:
np.shape(a1[0][2])

IndexError: index 2 is out of bounds for axis 0 with size 1

In [8]:
tx = np.concatenate((a1[0][0],a1[0][2]))

IndexError: index 2 is out of bounds for axis 0 with size 1

In [9]:
np.shape(a1[0][4])

IndexError: index 4 is out of bounds for axis 0 with size 1

In [10]:
b11 = np.zeros([len(a1[0][1]),5])

IndexError: index 1 is out of bounds for axis 0 with size 1

In [22]:
print((a1[0][4]))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 2 2 2 2 2 2
 2 2 2 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]


In [12]:
b1 = a1[0][1]

IndexError: index 1 is out of bounds for axis 0 with size 1

In [13]:
print(np.shape(b11))

NameError: name 'b11' is not defined

In [32]:
for i in range(len(b1)):
    b11[i][b1[i]] = 1

In [33]:
b11

array([[0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0.]])

In [36]:
Train_X = a1[0][0]
Train_Y = b11
Test_X = a1[0][2]
Test_Y = a1[0][3]

Test_Y_ori = a1[0][4]

In [48]:
with open('/home/sxz/data/geolife_Data/Augment_origin_data_Cross1.pickle', 'wb') as f:
    pickle.dump([Train_X, Train_Y, Test_X, Test_Y, Test_Y_ori], f)

In [42]:
with open('/home/sxz/data/geolife_Data/paper2_data_for_DL_kfold_dataset_RL.pickle', 'rb') as f:
    a1 , b1 = pickle.load(f)
print(np.shape(a1))
print(np.shape(b1))

(5, 5)
(4310, 1, 248, 4)


In [46]:
Test_X = a1[0][2]
Test_Y = a1[0][3]

Test_Y_ori = a1[0][4]

In [47]:
print(np.shape(Test_X))

(110, 1, 248, 4)


In [43]:
b11 = np.zeros([len(b1),5])

In [44]:
b11

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [45]:
for i in range(len(b1)):
    b11[i][b1[i]] = 1

IndexError: arrays used as indices must be of integer (or boolean) type

In [7]:
trainx = []
trainy =[]

In [8]:
len(b1[b1==0])
# print(np.where(b1==0))

random_b = np.random.choice(len(np.where(b1==0)[0]) , size=round(200), replace=False, p=None)

# print(random_b)

np.where(b1==0)[0][random_b]
train_index_ez=np.where(b1==0)[0][random_b]

# print(train_index_ez)

Train_X_tmp = a1[train_index_ez]
Train_Y_tmp = b11[train_index_ez]


# trainx.append(Train_X_tmp)
# trainy.append(Train_Y_

In [19]:
random_b = np.random.choice(len(np.where(b1==4)[0]) , size=round(200), replace=False, p=None)

# print(random_b)

train_index_ez=np.where(b1==4)[0][random_b]

# print(train_index_ez)

Train_X_tmp1 = a1[train_index_ez]
Train_Y_tmp1 = b11[train_index_ez]



In [20]:
Train_X_tmp = np.concatenate((Train_X_tmp,Train_X_tmp1),axis=0)
Train_Y_tmp = np.concatenate((Train_Y_tmp,Train_Y_tmp1))

In [31]:
Train_X_tmp = Train_X_tmp.reshape(650,1,31,128)
print(np.shape(Train_X_tmp))
print(np.shape(Train_Y_tmp))

(650, 1, 31, 128)
(650, 5)


In [32]:
Train_X = Train_X_tmp
Train_Y = Train_Y_tmp
Test_X = a1[test_index]
Test_Y = b11[test_index]

Test_Y_ori = b1[test_index]

In [33]:
with open('/home/sxz/data/geolife_Data/Averaged_Encoded_data_Cross.pickle', 'wb') as f:
    pickle.dump([Train_X, Train_Y, Test_X, Test_Y, Test_Y_ori], f)

In [23]:
b11

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]])

In [24]:
a1 = a1.reshape(4285,1,31,128)

In [25]:
all_index = np.arange(0,len(a1))

In [26]:
all_index

array([   0,    1,    2, ..., 4282, 4283, 4284])

In [27]:
test_index = np.random.choice(all_index, size=round(0.2*len(all_index)), replace=False, p=None)

In [28]:
test_index

array([ 789, 2009, 1835, 3278, 3230, 4098, 3356,  729, 2446, 2110, 2826,
       4188, 3238,  619, 3322, 2055, 3062, 3559, 1839, 3072, 4211, 2771,
       1161, 2884, 1683, 3549, 2096, 3261, 3821, 3259, 3957, 3465, 2773,
       1042, 3539,  973,  595, 1792,  532, 1041,  677,  354, 1292, 4172,
       4185, 3475, 3810, 3249, 4155, 4194, 4000, 2929, 3343, 2580, 2297,
       3558, 4129,  505, 3866, 2101,  656, 2676, 2418, 2734, 2658,  253,
       3935, 2808,  376, 1934, 2968, 4082, 2017,  377,  493,  850,  582,
       1650, 3077, 2638,  130,  855, 2909, 3667, 2766, 3370, 2911, 2620,
       4231, 3766, 3138, 4050, 3396, 3052, 2611, 1015, 3277, 1105, 4151,
       2625, 1965, 3785, 2955, 2569, 3503, 3490, 3439, 3128, 1675,  603,
       3690, 1905, 1417, 2219,  219, 2851, 3550, 1722, 3812,  774, 3953,
       3049, 1910,  910, 2597, 3616, 1917, 1519, 1564, 3886, 1654, 1314,
         72, 1975,  331, 3240, 2958,  477, 3147, 3125, 3699, 1574,  667,
       2305, 3228, 4196,  124, 2175,  766, 1222,  9

In [12]:
train_index = np.delete(all_index,test_index)

In [13]:
Train_X = a1[train_index]
Train_Y = b11[train_index]
Test_X = a1[test_index]
Test_Y = b11[test_index]

Test_Y_ori = b1[test_index]

In [74]:
with open('/home/sxz/data/geolife_Data/Encoded_data_Cross.pickle', 'wb') as f:
    pickle.dump([Train_X, Train_Y, Test_X, Test_Y, Test_Y_ori], f)

In [75]:
print(np.shape(Train_X))

(3428, 1, 31, 128)


In [76]:
filename = '/home/sxz/data/geolife_Data/paper2_data_for_DL_kfold_dataset_RL.pickle'
with open(filename, 'rb') as f:
    kfold_dataset, X_unlabeled = pickle.load(f)

In [78]:
print(kfold_dataset[0][1])

[0 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 1 0 0 0 0 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 1 0 3 3 0 0 0 0 0 1
 1 1 3 0 1 1 1 4 4 4 4 4 2 2 2 2 2 2 2 3 3 0 0 0 0 0 0 1 1 3 3 4 1 1 0 0 0
 0 0 2 0 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0 3 2 0 0 0 2 2 2 2 2 2 0 0 1 0 0 1 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 3 0 0 1 2 0 3 2 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 2 2 2 2 2 2 0 1 2 2 2 0 3 3 0 1 1 4 4 4 1 0 0 0
 2 2 1 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 0 0 0 0 0 0 2 0 2 2 2 0 3 2 2 2 2 2 0 4 4 4 4 4 4 4 0]


In [80]:
print(kfold_dataset[0][4])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 2 2 2 2 2 2
 2 2 2 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]


In [83]:
origin_data = np.concatenate((kfold_dataset[0][1],kfold_dataset[0][4]))

In [90]:
for i in range(5):
    a = len(origin_data[origin_data==i])
    print("mode index: {}".format(i))
    print("number of {} : {}".format(i,a))
    print("percent of {} : {}".format(i,a/len(origin_data)))

mode index: 0
number of 0 : 122
percent of 0 : 0.22141560798548093
mode index: 1
number of 1 : 33
percent of 1 : 0.05989110707803993
mode index: 2
number of 2 : 46
percent of 2 : 0.08348457350272233
mode index: 3
number of 3 : 18
percent of 3 : 0.032667876588021776
mode index: 4
number of 4 : 332
percent of 4 : 0.6025408348457351


In [94]:
np.shape(b1)
for i in range(5):
    a = len(b1[b1==i])
    print("mode index: {}".format(i))
    print("number of {} : {}".format(i,a))
    print("percent of {} : {}".format(i,a/len(b1)))

mode index: 0
number of 0 : 752
percent of 0 : 0.17549591598599767
mode index: 1
number of 1 : 132
percent of 1 : 0.030805134189031504
mode index: 2
number of 2 : 227
percent of 2 : 0.052975495915986
mode index: 3
number of 3 : 54
percent of 3 : 0.012602100350058343
mode index: 4
number of 4 : 3120
percent of 4 : 0.7281213535589265
